# Forecast using Machine Learning 

A reasonable starting point will be to use any of the tree based algorithms. Random forests Regressor will make for a good start. 

Here we'll also experiment with different algorithms 

In [1]:
import os
import sys
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
#local imports

sys.path.append(os.path.abspath(os.path.join('..')))
from src.data.manipulate import Manipulate


In [7]:
train_df = pd.read_csv('../data/processed/train.csv', index_col='Date')
test_df = pd.read_csv('../data/clean/test.csv', index_col='Date')

In [9]:
train_df.head()

,DayOfWeek,Weekend,WeekDay,Sales,Open,Promo,StateHoliday,DaysAfterHoliday,DaysToHoliday,SchoolHoliday,...,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,Day,MonthTiming,Month,Year,DayOfYear,WeekOfYear
Date,,,,,,,,,,,,,,,,,,,,,
2013-01-01,-1.000476,-0.630672,0.630672,-1.499723,-2.210440,-0.7854,3.366183,-0.377145,-0.968911,2.144211,...,-1.001128,0.802373,0.810673,0.045131,-1.673123,-1.236292,-1.457193,-1.070616,-1.587114,-1.56689
2013-01-02,-0.499823,-0.630672,0.630672,-0.063331,0.452399,-0.7854,-0.159217,-0.377145,1.133477,2.144211,...,-1.001128,0.802373,0.810673,0.045131,-1.559327,-1.236292,-1.457193,-1.070616,-1.577273,-1.56689
2013-01-03,0.000831,-0.630672,0.630672,-0.375805,0.452399,-0.7854,-0.159217,-0.023410,1.100627,2.144211,...,-1.001128,0.802373,0.810673,0.045131,-1.445530,-1.236292,-1.457193,-1.070616,-1.567432,-1.56689
2013-01-04,0.501484,-0.630672,0.630672,-0.334505,0.452399,-0.7854,-0.159217,0.330325,1.067778,2.144211,...,-1.001128,0.802373,0.810673,0.045131,-1.331734,-1.236292,-1.457193,-1.070616,-1.557591,-1.56689
2013-01-05,1.002138,1.585611,-1.585611,-0.201775,0.452399,-0.7854,-0.159217,0.684060,1.034928,2.144211,...,-1.001128,0.802373,0.810673,0.045131,-1.217938,-1.236292,-1.457193,-1.070616,-1.547750,-1.56689


In [10]:
X_train = train_df
y_train = train_df['Sales']

In [8]:
# pipe = Pipeline([
#     ("model", RandomForestRegressor(n_estimators=100))
# ])

In [11]:
# pipe.fit(train_df)
regressor = RandomForestRegressor(n_estimators=10, random_state=0)
regressor.fit(X_train, y_train)


RandomForestRegressor(n_estimators=10, random_state=0)

In [13]:
regressor.predict(X_train)

array([-1.49972273, -0.06333085, -0.37580453, ...,  0.49018653,
        0.68343708,  0.75486706])